In [1]:
import tensorflow as tf
from tensorflow import keras

In [3]:
!pip install -q -U keras_tuner


[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [4]:
import keras_tuner as kt

In [5]:
(img_train,label_train),(img_test,label_test) = keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 1s 0us/step


In [6]:
#normalizacja
img_train = img_train.astype('float32')/255.0
img_test = img_test.astype('float32')/255.0

In [9]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28,28)))
    hp_units = hp.Int('units',min_value=32, max_value=512,step=32)
    model.add(keras.layers.Dense(units=hp_units,activation='relu'))
    model.add(keras.layers.Dense(10))

    hp_learning_rate = hp.Choice('learning_rate',values=[1e-2, 1e-3, 1e-4])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    return model

In [11]:
#utworzenie wystąpienia tunera i hypertuning

tuner = kt.Hyperband(model_builder,
                      objective='val_accuracy',
                      max_epochs=10,
                      factor=3,
                      directory='my_dir',
                      project_name='moj_kt')

In [12]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5)

In [ ]:
tuner.search(img_train,label_train,epochs=50,validation_split=0.2,callbacks=[stop_early])
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
            Wyszukiwanie hyperparametrów is zakończone. 
            Optymalna liczba neuronów: {best_hps.get('units')},
            Optmalny paramet learning rate: {best_hps.get('learning_rate')}.
""")

Trial 28 Complete [00h 01m 14s]
val_accuracy: 0.8542500138282776

Best val_accuracy So Far: 0.8901666402816772
Total elapsed time: 00h 11m 29s

Search: Running Trial #29

Value             |Best Value So Far |Hyperparameter
512               |320               |units
0.001             |0.001             |learning_rate
10                |10                |tuner/epochs
0                 |4                 |tuner/initial_epoch
0                 |2                 |tuner/bracket
0                 |2                 |tuner/round

Epoch 1/10
1500/1500 [==============================] - 32s 20ms/step - loss: 0.4937 - accuracy: 0.8246 - val_loss: 0.3966 - val_accuracy: 0.8536
Epoch 2/10
 721/1500 [=============>................] - ETA: 12s - loss: 0.3727 - accuracy: 0.8646

In [ ]:
#trening modelu
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train,label_train,epochs=50,validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch))+1
print(f'Najlesza Epoka: {(best_epoch,)}')